In [ ]:
#!pip install tensorflow-gpu==2.0.0b1

# Diagnosis with an auto-encoder

based on Arvin Singh Kushwaha https://towardsdatascience.com/how-to-make-an-autoencoder-2f2d99cd5103

Tareas a realizar:
- Integre diagnósticos (Tensorboard) para poder observar mejor el entrenamiento
- Haga ajustes al auto-encoder para mejorar el desempeño



In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Input, Flatten,\
                                    Reshape, LeakyReLU,\
                                    Activation, Dropout
from tensorflow.keras.models import Model, Sequential
from matplotlib import pyplot as plt
from IPython import display # If using IPython, Colab or Jupyter
import numpy as np

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train/255.0
x_test = x_test/255.0

## Don't use everything: just a fraction of all available data
idx=np.random.choice(x_train.shape[0],int(0.2*x_train.shape[0]))

x_train = x_train[idx,:,:]
y_train = y_train[idx]

print("Training data with {0} samples".format(x_train.shape[0]))
print("Validation data with {0} samples".format(x_test.shape[0]))

# Plot image data from x_train
plt.imshow(x_train[0], cmap = "gray")
plt.show()

In [ ]:
LATENT_SIZE = 16
encoder = Sequential([
    Flatten(input_shape = (28, 28)),
    Dense(128),
    LeakyReLU(),
    #Dropout(0.5),
    Dense(64),
    LeakyReLU(),
    #Dropout(0.5),
    Dense(LATENT_SIZE, activation="sigmoid"),
])

In [ ]:
decoder = Sequential([
    Dense(64, input_shape = (LATENT_SIZE,)),
    LeakyReLU(),
    #Dropout(0.5),
    Dense(128),
    LeakyReLU(),
    Dense(784),
    Activation("sigmoid"),
    Reshape((28, 28))
])

In [ ]:
img = Input(shape = (28, 28))
latent_vector = encoder(img)
output = decoder(latent_vector)
model = Model(inputs = img, outputs = output)
model.compile("nadam", loss = "MeanAbsoluteError")

In [ ]:
EPOCHS = 20
#Only do plotting if you have IPython, Jupyter, or using Colab
for epoch in range(EPOCHS):
    fig, axs = plt.subplots(4, 4, figsize=(4,4))
    rand = x_test[np.random.randint(0, 10000, 16)].reshape((4, 4, 1, 28, 28))
    
    #display.clear_output() # If you imported display from IPython
    
    for i in range(4):
        for j in range(4):
            axs[i, j].imshow(model.predict(rand[i, j])[0], cmap = "gray")
            axs[i, j].axis("off")
    
    plt.subplots_adjust(wspace = 0, hspace = 0)
    plt.show()
    print("-----------", "EPOCH", epoch, "-----------")
    model.fit(x_train, x_train, batch_size = 64)